In [1]:
import os
import time
import socket
from osgeo import ogr
import geopandas as gpd
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [2]:
minio_ip = socket.gethostbyname('minio')
spark = SparkSession. \
    builder. \
    appName("Python Spark S3"). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.executor.memory", "80g"). \
    config("spark.driver.memory", "80g"). \
    config('spark.dirver.maxResultSize', '5g'). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.hadoop.fs.s3a.endpoint', 'http://'+minio_ip+':9000'). \
    config("spark.hadoop.fs.s3a.access.key", "minio-access-key"). \
    config("spark.hadoop.fs.s3a.secret.key", "minio-secret-key"). \
    config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.datasyslab:geotools-wrapper:geotools-24.0'). \
    getOrCreate()
SedonaRegistrator.registerAll(spark)

True

In [4]:
censo_df = spark.read.format("csv").\
    option("delimiter", ",").\
    option("header", "true").\
    option("inferSchema","true").\
    load("s3a://csv-data/*.csv")

In [6]:
censo_df.select("ENTIDAD","MUN","LOC","POBTOT").show(10)

+-------+---+---+-------+
|ENTIDAD|MUN|LOC| POBTOT|
+-------+---+---+-------+
|      1|  0|  0|1425607|
|      1|  1|  0| 948990|
|      1|  1|  1| 863893|
|      1|  1|  1|   2237|
|      1|  1|  1|    170|
|      1|  1|  1|    198|
|      1|  1|  1|    198|
|      1|  1|  1|    202|
|      1|  1|  1|    157|
|      1|  1|  1|    167|
+-------+---+---+-------+
only showing top 10 rows

